In [13]:
#Code to generate the required digester size
import pandas as pd
import numpy as np
result = pd.read_csv("../analysis/Household_Survey_Analysis_with_LPG_equiv.csv") 
print(result.head())
# Constants
Y = 0.3  # m³/kg biogas yield per kg of VS
VS = 0.8  # kg VS per kg of waste
retention_time_days = 30  # days

  House ID  Household Size Primary Fuel  Fuel mass (KG)  Time taken (days)  \
0     W001               3          LPG            12.5                 45   
1     W002               4          LPG            12.5                 40   
2     W003               4          LPG            12.5                 42   
3     W004               1          LPG             3.0                 50   
4     W005               5          LPG            12.5                 30   

   Solid waste/day (Kg)  Daily Fuel Usage (KG/day)  \
0                   1.2                   0.277778   
1                   1.8                   0.312500   
2                   0.8                   0.297619   
3                   0.3                   0.060000   
4                   2.7                   0.416667   

   Gross Energy per day (MJ/day)  Useful Energy per day (MJ/day)  \
0                      12.805556                        7.043056   
1                      14.406250                        7.923438   
2 

In [24]:
total_solid_waste_per_day = result['Solid waste/day (Kg)'].sum()
total_biogas_per_day = total_solid_waste_per_day * VS * Y
print(f"Total Solid Waste: {total_solid_waste_per_day:.2f} kg/day")

#Total biogas yield from sum of daily kitchen waste
print(f"\nTotal biogas production potential from kitchen waste: {total_biogas_per_day:.2f} m^3/day")

Total Solid Waste: 36.50 kg/day

Total biogas production potential from kitchen waste: 8.76 m^3/day


In [ ]:
total_useful_energy = result['Useful Energy per day (MJ/day)'].sum()

print(f"Total Useful Energy Demand: {total_useful_energy:.2f} MJ/day")

# Factors for digester sizing
biogas_energy_content = 20 # MJ/m^3 biogas (as specified by user)
biogas_stove_efficiency = 0.55 # from literature 
digester_hydraulic_retention_time = 30 # from literature (days)
design_buffer = 2 # community safety factor 

def required_biogas_yield(total_useful_energy, biogas_energy_content, biogas_stove_efficiency):
    return total_useful_energy / (biogas_energy_content * biogas_stove_efficiency)

Wumba_biogas_need = required_biogas_yield(total_useful_energy, biogas_energy_content, biogas_stove_efficiency)
print(f"\nRequired daily biogas to meet energy demand: {Wumba_biogas_need:.2f} m^3/day")

# --- Incorporating other waste sources ---
#Estimate the daily amount and biogas potential of other waste sources:
#
# estimated_daily_cow_dung_kg = 50 # example value
# biogas_yield_cow_dung = 0.1 # m^3 biogas per kg of cow dung (example value)
# biogas_potential_cow_dung_m3_per_day = estimated_daily_cow_dung_kg * biogas_yield_cow_dung

# estimated_daily_meat_waste_kg = 10 # example value
# biogas_yield_meat_waste = 0.3 # m^3 biogas per kg of meat waste (example value)
# biogas_potential_meat_waste_m3_per_day = estimated_daily_meat_waste_kg * biogas_yield_meat_waste

# total_waste_biogas_potential_m3_per_day = (total_solid_waste_kg_per_day * biogas_yield_per_kg_waste) + biogas_potential_cow_dung_m3_per_day + biogas_potential_meat_waste_m3_per_day
# print(f"Total daily biogas potential from all waste sources: {total_waste_biogas_potential_m3_per_day:.2f} m^3/day")


# Calculate the required digester volume based on Hydraulic Retention Time (HRT)
# Required Digester Volume = Total Wet Waste Input per Day * Hydraulic Retention Time * Design Buffer
# Assuming a rough estimate of total wet waste input (solid waste + other sources) and converting kg to m^3 with an assumed density (adjust as needed)
estimated_total_wet_waste_input_kg_per_day = total_solid_waste_per_day # + estimated_daily_cow_dung_kg + estimated_daily_meat_waste_kg # include other sources here
assumed_wet_waste_density_kg_per_m3 = 1000 # example value (density of water, adjust for waste)
estimated_total_wet_waste_input_m3_per_day = estimated_total_wet_waste_input_kg_per_day / assumed_wet_waste_density_kg_per_m3

required_digester_volume_m3_hrt = estimated_total_wet_waste_input_m3_per_day * digester_hydraulic_retention_time * design_buffer

print(f"Required biogas digester volume: {required_digester_volume_m3_hrt:.2f} m^3")

# Compare the total biogas potential with the required yield to see if it's sufficient.
# print(f"\nIs total waste biogas potential sufficient? {total_waste_biogas_potential_m3_per_day >= required_biogas_yield_m3_per_day}")

#Currently, the biogas potential from other waste sources is not included in the sufficiency check.


Total Useful Energy Demand: 233.61 MJ/day

Required daily biogas to meet energy demand: 21.24 m^3/day
Required biogas digester volume: 2.19 m^3
